In [ ]:
%pip install PyCryptodome

In [ ]:
from Crypto.Util.number import inverse
import math

In [ ]:
N = 14905562257842714057932724129575002825405393502650869767115942606408600343380327866258982402447992564988466588305174271674657844352454543958847568190372446723549627752274442789184236490768272313187410077124234699854724907039770193680822495470532218905083459730998003622926152590597710213127952141056029516116785229504645179830037937222022291571738973603920664929150436463632305664687903244972880062028301085749434688159905768052041207513149370212313943117665914802379158613359049957688563885391972151218676545972118494969247440489763431359679770422939441710783575668679693678435669541781490217731619224470152467768073
e1 = 12886657667389660800780796462970504910193928992888518978200029826975978624718627799215564700096007849924866627154987365059524315097631111242449314835868137
e2 = 12110586673991788415780355139635579057920926864887110308343229256046868242179445444897790171351302575188607117081580121488253540215781625598048021161675697
c1 = 14010729418703228234352465883041270611113735889838753433295478495763409056136734155612156934673988344882629541204985909650433819205298939877837314145082403528055884752079219150739849992921393509593620449489882380176216648401057401569934043087087362272303101549800941212057354903559653373299153430753882035233354304783275982332995766778499425529570008008029401325668301144188970480975565215953953985078281395545902102245755862663621187438677596628109967066418993851632543137353041712721919291521767262678140115188735994447949166616101182806820741928292882642234238450207472914232596747755261325098225968268926580993051
c2 = 14386997138637978860748278986945098648507142864584111124202580365103793165811666987664851210230009375267398957979494066880296418013345006977654742303441030008490816239306394492168516278328851513359596253775965916326353050138738183351643338294802012193721879700283088378587949921991198231956871429805847767716137817313612304833733918657887480468724409753522369325138502059408241232155633806496752350562284794715321835226991147547651155287812485862794935695241612676255374480132722940682140395725089329445356434489384831036205387293760789976615210310436732813848937666608611803196199865435145094486231635966885932646519

We remember that $(a+b)^n$ is a sum of elements of the form $a^kb^{n-k}$, where $k$ is an integer:
$$(a*q+b*q)^{e_1} = \sum_i constant(i)*q^{i}p^{e_1-i}$$
But as our equations are modulo $N=p*q$ only the elements for which $i=0$ or $i=e_1$ will survive. So in fact we look at the two equations:
$$\begin{aligned}
c_1 &= 2^{e_1}*p^{e_1} + 3^{e_1}*q^{e_1} \pmod N \\
c_2 &= 5^{e_2}*p^{e_2} + 7^{e_2}*q^{e_2} \pmod N
\end{aligned}$$
As the trick we used works no matter what the power is, we can raise the first equation to the power $e_2$ and the second to the power $e_1$:
$$\begin{aligned}
{c_1}^{e_2} &= 2^{e_1e_2}*p^{e_1e_2} + 3^{e_1e_2}*q^{e_1e_2} \pmod N \\
{c_2}^{e_1} &= 5^{e_1e_2}*p^{e_1e_2} + 7^{e_1e_2}*q^{e_1e_2} \pmod N
\end{aligned}$$
Now we have to linear equations in $u=p^{e_1e_2}$ and $v=q^{e_1e_2}$, can be solved:
$$\begin{aligned}
{c_1}^{e_2} * inv(2^{e_1e_2},N) &= u + 3^{e_1e_2}*v \pmod N \\
{c_2}^{e_1} * inv(5^{e_1e_2},N) &= u + 7^{e_1e_2}*v \pmod N
\end{aligned}$$
so
$${c_1}^{e_2} * inv(2^{e_1e_2},N) - {c_2}^{e_1} * inv(5^{e_1e_2},N) = (3^{e_1e_2} - 7^{e_1e_2}) * v \pmod N$$

In [ ]:
v = (inverse(pow(3,e1*e2,N),N) - inverse(pow(7,e1*e2,N),N)) * (pow(c1,e2,N)*inverse(pow(2,e1*e2,N),N) - pow(c2,e1,N)*inverse(pow(5,e1*e2,N),N)) % N
v

As we know the value of $q^{e_1e_2} \pmod N$ we can write it as:
$$ q * q^{e_1e_2-1} = A + B * N \Rightarrow A = q * (q^{e_1e_2-1} - B * p)$$
where $A$ and $B$ are integers. This means that $q$ is a factor of $A$ and so we can find it by computing the gcd of $A$ and $N=p*q$.

In [ ]:
q = math.gcd(v, N)
p = N // q
p, q